In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
df = pd.read_csv('model_results.csv')
df

,x,y,width,height,confidence,class,image_path,prediction_type
0,783,385,429,635,0.568012,infested_tree,jpg/tile_0_15.jpg,ObjectDetectionModel
1,91,413,182,354,0.523075,infested_tree,jpg/tile_1_15.jpg,ObjectDetectionModel
2,174,578,345,423,0.523283,infested_tree,jpg/tile_2_12.jpg,ObjectDetectionModel
3,912,269,171,539,0.289279,infested_tree,jpg/tile_2_13.jpg,ObjectDetectionModel
4,112,722,225,545,0.504943,infested_tree,jpg/tile_3_14.jpg,ObjectDetectionModel
...,...,...,...,...,...,...,...,...
193,409,804,487,390,0.683156,infested_tree,jpg/tile_25_5.jpg,ObjectDetectionModel
194,703,664,337,296,0.508543,healthy_tree,jpg/tile_25_6.jpg,ObjectDetectionModel
195,174,160,348,321,0.436400,infested_tree,jpg/tile_25_6.jpg,ObjectDetectionModel
196,67,867,132,264,0.258431,healthy_tree,jpg/tile_25_6.jpg,ObjectDetectionModel


In [3]:
df.drop('prediction_type', axis='columns', inplace=True)
df

,x,y,width,height,confidence,class,image_path
0,783,385,429,635,0.568012,infested_tree,jpg/tile_0_15.jpg
1,91,413,182,354,0.523075,infested_tree,jpg/tile_1_15.jpg
2,174,578,345,423,0.523283,infested_tree,jpg/tile_2_12.jpg
3,912,269,171,539,0.289279,infested_tree,jpg/tile_2_13.jpg
4,112,722,225,545,0.504943,infested_tree,jpg/tile_3_14.jpg
...,...,...,...,...,...,...,...
193,409,804,487,390,0.683156,infested_tree,jpg/tile_25_5.jpg
194,703,664,337,296,0.508543,healthy_tree,jpg/tile_25_6.jpg
195,174,160,348,321,0.436400,infested_tree,jpg/tile_25_6.jpg
196,67,867,132,264,0.258431,healthy_tree,jpg/tile_25_6.jpg


In [25]:
def get_tile_col(row):
    return int(row.image_path.split('_')[1])

def get_tile_row(row):
    return int(row.image_path.replace('.jpg', '').split('_')[2])  

def get_object_centroid_x(row):
    return row.x + row.width // 2

def get_object_centroid_y(row):
    return row.y - row.height // 2    

def get_geotransform(row):
    xml_path = row.image_path + '.aux.xml'
    xml_data = open(xml_path, 'r').read()
    root = ET.XML(xml_data) # Parse XML
    gt_str = root.find('GeoTransform').text
    gt_str_list = gt_str.split(',')
    gt_list = [float(x) for x in gt_str_list]       
    return gt_list    

def get_transformed_bb(row):
    """
    GT(0) x-coordinate of the upper-left corner of the upper-left pixel.
    GT(1) w-e pixel resolution / pixel width.
    GT(2) row rotation (typically zero).
    GT(3) y-coordinate of the upper-left corner of the upper-left pixel.
    GT(4) column rotation (typically zero).
    GT(5) n-s pixel resolution / pixel height (negative value for a north-up image).
    
    POLYGON (( 10 10, 10 20, 20 20, 20 15, 10 10))
    clocwise
    """
    gt = row.geotransform
    ulx = gt[0] + row.x * gt[1]
    uly = gt[3] + row.y * gt[5]
    lrx = ulx + row.width * gt[1]
    lry = uly + row.height * gt[5]
    wkt = f'POLYGON (({ulx} {uly}, {ulx} {lry}, {lrx} {lry}, {lrx} {uly}, {ulx} {uly}))'
    return wkt

In [26]:
df['tile_col'] = df.apply(get_tile_col, axis=1)
df['tile_row'] = df.apply(get_tile_row, axis=1)
df['object_centroid_x'] = df.apply(get_object_centroid_x, axis=1)
df['object_centroid_y'] = df.apply(get_object_centroid_y, axis=1)
df['geotransform'] = df.apply(get_geotransform, axis=1)
df['bb'] = df.apply(get_transformed_bb, axis=1)
df

,x,y,width,height,confidence,class,image_path,tile_col,tile_row,object_centroid_x,object_centroid_y,geotransform,mylist,bb
0,783,385,429,635,0.568012,infested_tree,jpg/tile_0_15.jpg,0,15,997,68,"[259821.30138992154, 0.014080873475957724, 0.0...",1.1,POLYGON ((259832.32671385322 1483933.807200336...
1,91,413,182,354,0.523075,infested_tree,jpg/tile_1_15.jpg,1,15,182,236,"[259835.3822633975, 0.014080873475957724, 0.0,...",1.1,POLYGON ((259836.66362288382 1483933.412935879...
2,174,578,345,423,0.523283,infested_tree,jpg/tile_2_12.jpg,2,12,346,367,"[259849.46313687347, 0.014080873475957724, 0.0...",1.1,POLYGON ((259851.91320885828 1483973.332212183...
3,912,269,171,539,0.289279,infested_tree,jpg/tile_2_13.jpg,2,13,997,0,"[259849.46313687347, 0.014080873475957724, 0.0...",1.1,POLYGON ((259862.30489348355 1483963.602328611...
4,112,722,225,545,0.504943,infested_tree,jpg/tile_3_14.jpg,3,14,224,450,"[259863.5440103494, 0.014080873475957724, 0.0,...",1.1,POLYGON ((259865.1210681787 1483943.1428194512...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,409,804,487,390,0.683156,infested_tree,jpg/tile_25_5.jpg,25,5,652,609,"[260173.32322682047, 0.014080873475957724, 0.0...",1.1,POLYGON ((260179.08230407213 1484068.716049109...
194,703,664,337,296,0.508543,healthy_tree,jpg/tile_25_6.jpg,25,6,871,516,"[260173.32322682047, 0.014080873475957724, 0.0...",1.1,POLYGON ((260183.22208087408 1484056.606497920...
195,174,160,348,321,0.436400,infested_tree,jpg/tile_25_6.jpg,25,6,348,0,"[260173.32322682047, 0.014080873475957724, 0.0...",1.1,POLYGON ((260175.7732988053 1484063.7032581524...
196,67,867,132,264,0.258431,healthy_tree,jpg/tile_25_6.jpg,25,6,133,735,"[260173.32322682047, 0.014080873475957724, 0.0...",1.1,POLYGON ((260174.26664534336 1484053.748080605...


In [28]:
df.to_csv('bounding_boxes.csv', index=False)
print('Dataframe stored in bounding_boxes.csv')
print('FINISHED')

Dataframe stored in bounding_boxes.csv
FINISHED


In [7]:
for i in range(0, 6):
    print(df.geotransform[i])

  2.5982130138992154e+05,  1.4080873475957724e-02,  0.0000000000000000e+00,  1.4839392283366250e+06,  0.0000000000000000e+00, -1.4080873475957724e-02
  2.5983538226339751e+05,  1.4080873475957724e-02,  0.0000000000000000e+00,  1.4839392283366250e+06,  0.0000000000000000e+00, -1.4080873475957724e-02
  2.5984946313687347e+05,  1.4080873475957724e-02,  0.0000000000000000e+00,  1.4839814709570529e+06,  0.0000000000000000e+00, -1.4080873475957724e-02
  2.5984946313687347e+05,  1.4080873475957724e-02,  0.0000000000000000e+00,  1.4839673900835768e+06,  0.0000000000000000e+00, -1.4080873475957724e-02
  2.5986354401034940e+05,  1.4080873475957724e-02,  0.0000000000000000e+00,  1.4839533092101009e+06,  0.0000000000000000e+00, -1.4080873475957724e-02
  2.5986354401034940e+05,  1.4080873475957724e-02,  0.0000000000000000e+00,  1.4839392283366250e+06,  0.0000000000000000e+00, -1.4080873475957724e-02


In [8]:
df.geotransform[0]

'  2.5982130138992154e+05,  1.4080873475957724e-02,  0.0000000000000000e+00,  1.4839392283366250e+06,  0.0000000000000000e+00, -1.4080873475957724e-02'

In [9]:
image_path = 'jpg/tile_0_15.jpg'
xml_path = image_path + '.aux.xml'
xml_data = open(xml_path, 'r').read()  # Read file
root = ET.XML(xml_data) # Parse XML
gt_str = root.find('GeoTransform').text
gt_str_list = gt_str.split(',')
gt = [float(x) for x in gt_str_list]
gt




[259821.30138992154,
 0.014080873475957724,
 0.0,
 1483939.228336625,
 0.0,
 -0.014080873475957724]